#### Домашняя работа Урок 4. Рекоммендательные системы на основе контента. Поиск похожих товаров и пользователей

#### Варианты для ранжирования (Краткое введение. Подробности - на следующем вебинаре)

**Вариант 1.**  
*As is* - ранжировать по output ALS

**Вариант 2.**
*Та же модель, другой лосс - BPR*

BPR - Bayesian Personalized Ranking loss
1. Случайная пара user - купленный им item_i = позитивная пара
2. Для того же юзера сэмплируем item_j (не купил или вес в user-item матрице ниже, у item_i из 1.) - негативная пара
3. Прогнозы модели $p_{ui}$, $p_{uj}$ --> $p_{ui} - p_{uj}$
4. loss = $\sigma(p_{ui} - p_{uj})$ - это и есть наша ошибка
5. Обновляем эмбеддинги

### 1. Базовое применение  BPR - Bayesian Personalized Ranking loss

In [1]:
!pip install watermark

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Для работы с матрицами
from scipy.sparse import csr_matrix


# Bayesian Personalized Ranking loss
from implicit.bpr import BayesianPersonalizedRanking

from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Матричная факторизация
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Метрики
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))

if module_path not in sys.path:
    sys.path.append(module_path)

import watermark
# from src.metrics import precision_at_k, recall_at_k

### 1.1 Загрузим данные по проданным товарам

In [3]:
data = pd.read_csv('./data/retail_train.csv')

In [4]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

### 1.2 Train-test split

In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [7]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


### 1.3 Загрузим данные информации о товарах

In [8]:
item_features = pd.read_csv('./data/product.csv')

In [9]:
# item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

*Результаты тестовых рекомендаций запишем в массив result*

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [14]:
data_train.shape

(2278490, 12)

In [15]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


### 1.4 Подготовим матрицы и данные для ввода в модель

##### 1.4.1 Построим:
- сводную матрицу user_item_matrix, где строки 'user_id', столбцы 'item_id', в ячейках 0 или 1;
- переведем ее с формат saprse matrix;
- используем ее для построения всех моделей.

In [18]:
%%time

# Заведем фиктивный item_id (если юзер покупал товары НЕ из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1  
user_item_matrix = user_item_matrix.astype(float) 

# переведем в формат saprse matrix
# sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

CPU times: user 1.19 s, sys: 585 ms, total: 1.78 s
Wall time: 2.26 s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*Посмотрим размерность матрицы user_item_matrix и % ее заполнения*

In [19]:
user_item_matrix.shape

(2499, 5001)

#### Для обучения модли мы должны на вход подать разреженную матрицу sparse_user_item, где строки user а столбцы item. Здесь важен порядок. 

#### Переведем user_item_matrix в формат saprse matrix

In [20]:
%%time
# переведем user_item_matrix в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

CPU times: user 137 ms, sys: 79.6 ms, total: 216 ms
Wall time: 325 ms


##### 1.4.2 Построим:
- сводную матрицу user_item_matrix_test, где строки 'user_id', столбцы 'item_id', в ячейках 0 или 1;
- переведем ее в формат saprse matrix;
- используем ее для построения метрик всех моделей.

In [23]:
%%time

# Заведем фиктивный item_id (если юзер покупал товары НЕ из топ-5000, то он "купил" такой товар)
data_test.loc[~data_test['item_id'].isin(top_5000), 'item_id'] = 999999
# data_test !

user_item_matrix_test = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix_test[user_item_matrix_test > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix_test = user_item_matrix_test.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
# sparse_user_item_test = csr_matrix(user_item_matrix).tocsr()

user_item_matrix_test.head(3)

CPU times: user 810 ms, sys: 457 ms, total: 1.27 s
Wall time: 1.8 s


item_id,397896,480014,707683,818980,819063,819227,819255,819304,819308,819330,...,15596279,15596488,15596515,15831255,15926712,15926775,15926844,15926886,15927661,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
user_item_matrix_test.shape

(2042, 4637)

In [25]:
# % заполнения user_item_matrix_test
user_item_matrix_test.sum().sum() / (user_item_matrix_test.shape[0] * user_item_matrix_test.shape[1]) * 100

0.671292125658772

In [26]:
%%time

# переведем в формат saprse matrix
sparse_user_item_test = csr_matrix(user_item_matrix_test).tocsr()

CPU times: user 116 ms, sys: 72.1 ms, total: 188 ms
Wall time: 341 ms


##### Подготовим дополнительные данные для формирования результатов и  работы с результатами рекомендаций

In [27]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 2. Создаем и обучаем модели

### 2.1 Создаем и обучаем модель ItemItemRecommender. Базовый уровень.

In [28]:
%%time

model_ItemItem = ItemItemRecommender(K=25, num_threads=4)

CPU times: user 0 ns, sys: 6 µs, total: 6 µs
Wall time: 8.11 µs


In [29]:
%%time

model_ItemItem.fit(sparse_user_item, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 831 ms, sys: 393 ms, total: 1.22 s
Wall time: 3.76 s


In [30]:
%%time
# введем порядковый номер user, начинается с 1
user_n = 1
id_n, scores_n = model_ItemItem.recommend(userid=userid_to_id[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5,
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

CPU times: user 0 ns, sys: 6 ms, total: 6 ms
Wall time: 15.3 ms


In [31]:
id_n, scores_n

(array([2381, 3408, 2148, 3587,  300], dtype=int32),
 array([78679., 72173., 60612., 53023., 52387.]))

In [32]:
res_n = [id_to_itemid[rec] for rec in id_n]
res_n

[999999, 1082185, 981760, 1098066, 840361]

In [33]:
%%time
precision_at_k(model_ItemItem, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

CPU times: user 191 ms, sys: 111 ms, total: 302 ms
Wall time: 605 ms


0.010022459292532286

### 2.2 Создаем и обучаем модель BayesianPersonalizedRanking

In [34]:
%%time

model_BPR = BayesianPersonalizedRanking(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                num_threads=4,
                                random_state=42)

model_BPR.fit(sparse_user_item, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 3.19 s, sys: 1.87 s, total: 5.06 s
Wall time: 6.89 s


##### Сделаем предсказание для одного user. BayesianPersonalizedRanking

In [35]:
%%time
# введем порядковый номер user, начинается с 0
# обращаем внимание на ввод userid=userids[user_n]
user_n = 2
id_BPR_n, score_BPR_n = model_BPR.recommend(userid=userids[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None)

CPU times: user 0 ns, sys: 10.6 ms, total: 10.6 ms
Wall time: 27.1 ms


In [36]:
id_BPR_n, score_BPR_n

(array([2381, 3408, 2148, 3587,  110], dtype=int32),
 array([5.1828136, 2.5046372, 1.49694  , 1.3888223, 1.3335671],
       dtype=float32))

In [37]:
res_BPR_n = [id_to_itemid[rec] for rec in id_BPR_n]
res_BPR_n

[999999, 1082185, 981760, 1098066, 826249]

##### Сделаем предсказание для всех user. Model BayesianPersonalizedRanking. Данный метод располагает результаты рекомендаций по убыванию, это видно по score, расположенных по убыванию

In [38]:
matrix_userids

array([   0,    1,    2, ..., 2496, 2497, 2498])

In [39]:
%%time

ids_BPR_all, scores_BPR_all = \
                model_BPR.recommend(userid=matrix_userids,
                N=5,
                user_items=sparse_user_item[matrix_userids])

CPU times: user 203 ms, sys: 212 ms, total: 415 ms
Wall time: 554 ms


In [40]:
ids_BPR_all, scores_BPR_all

(array([[3947, 1927,  503,  832, 2696],
        [2148, 1927, 2307, 4346,  832],
        [3557, 2148, 3200, 3173, 2757],
        ...,
        [3947, 2148, 1927, 2307,  503],
        [4016, 2757, 2307, 1438,  731],
        [ 503,  832,  575, 2696, 1135]], dtype=int32),
 array([[2.2938132 , 1.8025146 , 1.7848451 , 1.6727738 , 1.51284   ],
        [1.3972294 , 1.1248392 , 1.1216831 , 1.0194914 , 0.99705434],
        [1.9171375 , 1.7552001 , 1.6890769 , 1.6722965 , 1.6412863 ],
        ...,
        [1.5534906 , 1.3631874 , 1.1653608 , 1.1089916 , 1.0855249 ],
        [1.7828512 , 1.5033679 , 1.3940239 , 1.3174914 , 1.3007056 ],
        [1.3759646 , 1.3074389 , 1.2652845 , 1.138861  , 1.0971358 ]],
       dtype=float32))

In [41]:
# преобразуем данные для рекомендации user [2]
res_BPR_all = [id_to_itemid[rec] for rec in ids_BPR_all[2]]
res_BPR_all

[1096036, 981760, 1065593, 1062966, 1029743]

!!! Ниже приведенный код не заработал. После введения вместо  userid=userid_to_id другую адресацию юзеров userid=matrix_userids Все заработало!

In [42]:
# Ниже приведенный код не заработал!
# %%time

# ids_BPR_all, scores_BPR_all = model_BPR.recommend(userid=userid_to_id,
#                         user_items=sparse_user_item,
#                         N=5, 
#                         filter_already_liked_items=False, 
#                         filter_items=None)

Каждая модель в implicit также имеет возможность отображать связанные элементы с помощью метода similar_items. Например, чтобы приобрести сопутствующие товары для itemid:

Вычислим список похожих элементов на itemid в колчестве N=5

In [43]:
itemid=19

itemids_BPR_top, scores_BPR_items_top= model_BPR.similar_items(itemid =itemid, N=5)

In [44]:
itemids_BPR_top, scores_BPR_items_top

(array([  19, 1026, 1324, 4186,  787], dtype=int32),
 array([0.99999994, 0.9523099 , 0.95127386, 0.9506359 , 0.9503067 ],
       dtype=float32))

Вычислим список похожих элементов на userid=19 в колчестве N=5. Первый элемент в списке будет сам userid=19. Список ранжирован.

In [45]:
userid=19

user_BPR_top, scores_BPR_user_top= model_BPR.similar_users(userid =userid, N=5)

In [46]:
user_BPR_top, scores_BPR_user_top

(array([  19, 1413, 2493,  309, 1359], dtype=int32),
 array([0.9999999, 0.9966899, 0.9965342, 0.9965156, 0.9963294],
       dtype=float32))

#### precision_at_k

In [47]:
%%time
precision_at_k(model_BPR, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

CPU times: user 241 ms, sys: 214 ms, total: 455 ms
Wall time: 855 ms


0.009629421673217294

# Выводы:
- Модель на основе BayesianPersonalizedRanking заработала;
- результаты метрик precision_at_k хуже чем у модели ItemItemRecommender, соответственно 0.001280 против 0.001649;
- применять модель на основе BayesianPersonalizedRanking надо, и в зависимости от требуемого быстродействия и точности она может быть предпрочтительнее других моделей.

### 2.2 Создаем и обучаем модель AlternatingLeastSquares

In [48]:
%%time

model_ALS = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
#                                     alpha=2.0,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                   random_state=42)


model_ALS.fit(sparse_user_item, show_progress=True)

/home/krassa/anaconda3/lib/python3.9/site-packages/implicit/utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 3.99 s, sys: 3.17 s, total: 7.16 s
Wall time: 10.9 s


##### Сделаем предсказание для одного user. Model AlternatingLeastSquares

In [49]:
%%time
# введем порядковый номер user, начинается с 0
# обращаем внимание на ввод userid=userids[user_n]
user_n = 2
id_ALS_n, score_ALS_n = model_ALS.recommend(userid=userids[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

CPU times: user 1.85 ms, sys: 36.1 ms, total: 38 ms
Wall time: 86.7 ms


In [50]:
id_ALS_n, score_ALS_n

(array([4016, 1263, 3688, 4598, 1639], dtype=int32),
 array([1.3204186, 1.2811381, 1.2744088, 1.1281989, 1.0828598],
       dtype=float32))

In [51]:
res_ALS_n = [id_to_itemid[rec] for rec in id_ALS_n]
res_ALS_n

[1133018, 910032, 1106523, 9337581, 938700]

##### Сделаем предсказание для всех user. Model AlternatingLeastSquares. Данный метод располагает результаты рекомендаций по убыванию, это видно по score, расположенных по убыванию

In [52]:
%%time

ids_ALS_all, scores_ALS_all = model_ALS.recommend(userid=userid_to_id,
                        user_items=sparse_user_item,
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

CPU times: user 614 ms, sys: 324 ms, total: 938 ms
Wall time: 1.38 s


In [53]:
ids_ALS_all, scores_ALS_all

(array([[2307, 2788, 3408, 2381, 2440],
        [3230, 4016, 3688, 4157, 3062],
        [4016, 1263, 3688, 4598, 1639],
        ...,
        [3062,  300, 4643,  558, 1350],
        [3587, 4346, 2302,  110, 2927],
        [3133, 2381, 3408, 2316, 2148]], dtype=int32),
 array([[1.115137  , 0.94127184, 0.8930241 , 0.85336757, 0.8382009 ],
        [1.0864737 , 1.0539333 , 1.0246016 , 1.0118954 , 0.9801687 ],
        [1.3204188 , 1.2811376 , 1.2744088 , 1.1281987 , 1.08286   ],
        ...,
        [1.0787325 , 1.0557972 , 0.9364862 , 0.9167248 , 0.88197386],
        [1.2400308 , 1.1537303 , 1.1464189 , 1.1186818 , 1.022287  ],
        [1.258044  , 1.2518576 , 1.1879405 , 1.1262484 , 1.0103698 ]],
       dtype=float32))

In [54]:
# преобразуем данные для рекомендации user [2]
res_ALS_all = [id_to_itemid[rec] for rec in ids_ALS_all[2]]
res_ALS_all

[1133018, 910032, 1106523, 9337581, 938700]

Каждая модель в implicit также имеет возможность отображать связанные элементы с помощью метода similar_items. Например, чтобы приобрести сопутствующие товары для itemid (the Beatles):

Вычислим список похожих элементов на itemid в колчестве N=5

In [55]:
itemid=19

itemids_ALS_top, scores_ALS_items_top= model_ALS.similar_items(itemid =itemid, N=5)

In [56]:
itemids_ALS_top, scores_ALS_items_top

(array([  19, 2743, 4571, 1023, 4891], dtype=int32),
 array([0.99999994, 0.67235786, 0.6501836 , 0.6356612 , 0.63506013],
       dtype=float32))

In [57]:
%%time
precision_at_k(model_ALS, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

CPU times: user 211 ms, sys: 179 ms, total: 389 ms
Wall time: 786 ms


0.00884334643458731

### 2.3 Создаем и обучаем модель AlternatingLeastSquares с улучшением tfidf и bm25 взвешиваним 

##### Улучшение модели:

Первым шагом в использовании этой модели будет преобразование необработанных показателей взаимодействиям из исходного набора данных в значения, которые можно использовать в качестве конфиденциальных данных. Придадим повторным воспроизведениям больше уверенности в модели, но сделать так, чтобы этот эффект уменьшался по мере увеличения количества повторных воспроизведений, чтобы уменьшить влияние одного суперпокупателя на модель. Аналогичным образом, уменьшим часть веса доверия от популярных товаров. Для этого мы будем использовать схему взвешивания tfidf и bm25, используемым классическим поиском информации:

### 2.4 Метрика precision_at_k

In [66]:
%%time
precision_at_k(model_ItemItem, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

CPU times: user 188 ms, sys: 163 ms, total: 350 ms
Wall time: 816 ms


0.010022459292532286